In [75]:
import sys 
import os
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.features import preprocessing
from pathlib import Path
import pandas as pd

In [76]:
data_path = Path('/home/ralmasri/projects/Thesis/Domain-Guided-Monitoring/data/')
csv_path = data_path / "logs_aggregated_concurrent.csv"
size = 50
# data_df = pd.read_csv(csv_path).head(size) if size > 0  else pd.read_csv(csv_path)
subset_path = data_path / f"{size}_logs_aggregated_concurrent.csv" if size > 0 else csv_path
# if size > 0:
#     data_df.to_csv(subset_path)

In [77]:
huawei_config = preprocessing.HuaweiPreprocessorConfig()
huawei_config.aggregated_log_file = subset_path
sequence_preprocessor = preprocessing.ConcurrentAggregatedLogsPreprocessor(huawei_config)
sequence_column_name = sequence_preprocessor.sequence_column_name
# data_df = pd.read_csv(data_path / "logs_preprocessed.csv")
# x = data_df['attributes'].iloc[0]
# import ast
# x = ast.literal_eval(x)

In [78]:
log_df = sequence_preprocessor._load_log_only_data()
log_df.head()

Generating DRAIN clusters from log_df: 100%|██████████| 15/15 [00:00<00:00, 1840.79it/s]


,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,fine_log_cluster_template,fine_log_cluster_path,coarse_log_cluster_template,coarse_log_cluster_path,timestamp,url_cluster_template,url_cluster_path
0,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:00:48.274000000+01:00,"130.149.249.127 ""GET /v2.0/networks?shared=Tru...",,get v20 networks shared true http 11 status 20...,13 get v20 networks shared true http * status *,get v20 networks shared true http 11 status 20...,13 get v20 networks,2019-11-19 17:00:48.274000+01:00,,NaN
1,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:01:50.119000000+01:00,"130.149.249.127 ""GET /v2.0/ports?fields=bindin...",,get v20 ports fields binding 3ahost id fields ...,22 get v20 ports fields binding * id fields bi...,get v20 ports fields binding 3ahost id fields ...,22 get v20 ports,2019-11-19 17:01:50.119000+01:00,,NaN
2,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:01:50.210000000+01:00,"130.149.249.127 ""GET /v2.0/ports?tenant_id=99c...",,get v20 ports tenant id 99c2677b197747c9bd8bc0...,17 get v20 ports tenant id * device id *,get v20 ports tenant id 99c2677b197747c9bd8bc0...,17 get v20 ports,2019-11-19 17:01:50.210000+01:00,,NaN
3,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:01:50.317000000+01:00,"130.149.249.127 ""GET /v2.0/networks?shared=Tru...",,get v20 networks shared true http 11 status 20...,13 get v20 networks shared true http * status *,get v20 networks shared true http 11 status 20...,13 get v20 networks,2019-11-19 17:01:50.317000+01:00,,NaN
4,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:02:52.398000000+01:00,"130.149.249.127 ""GET /v2.0/networks?tenant_id=...",,get v20 networks tenant id 99c2677b197747c9bd8...,16 get v20 networks tenant id * shared false http,get v20 networks tenant id 99c2677b197747c9bd8...,16 get v20 networks,2019-11-19 17:02:52.398000+01:00,,NaN


In [79]:
log_df['grouper'] = 1
log_df.head(3)

,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,fine_log_cluster_template,fine_log_cluster_path,coarse_log_cluster_template,coarse_log_cluster_path,timestamp,url_cluster_template,url_cluster_path,grouper
0,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:00:48.274000000+01:00,"130.149.249.127 ""GET /v2.0/networks?shared=Tru...",,get v20 networks shared true http 11 status 20...,13 get v20 networks shared true http * status *,get v20 networks shared true http 11 status 20...,13 get v20 networks,2019-11-19 17:00:48.274000+01:00,,NaN,1
1,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:01:50.119000000+01:00,"130.149.249.127 ""GET /v2.0/ports?fields=bindin...",,get v20 ports fields binding 3ahost id fields ...,22 get v20 ports fields binding * id fields bi...,get v20 ports fields binding 3ahost id fields ...,22 get v20 ports,2019-11-19 17:01:50.119000+01:00,,NaN,1
2,wally113,INFO,neutron-server,neutron.wsgi,,,2019-11-19T17:01:50.210000000+01:00,"130.149.249.127 ""GET /v2.0/ports?tenant_id=99c...",,get v20 ports tenant id 99c2677b197747c9bd8bc0...,17 get v20 ports tenant id * device id *,get v20 ports tenant id 99c2677b197747c9bd8bc0...,17 get v20 ports,2019-11-19 17:01:50.210000+01:00,,NaN,1


In [80]:
for column in sequence_preprocessor.relevant_columns:
    log_df[column] = log_df[column].apply(
        lambda x: column + "#" + x.lower() if len(x) > 0 else ""
    )
log_df.head(3)

,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,fine_log_cluster_template,fine_log_cluster_path,coarse_log_cluster_template,coarse_log_cluster_path,timestamp,url_cluster_template,url_cluster_path,grouper
0,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:00:48.274000000+01:00,"130.149.249.127 ""GET /v2.0/networks?shared=Tru...",,fine_log_cluster_template#get v20 networks sha...,13 get v20 networks shared true http * status *,coarse_log_cluster_template#get v20 networks s...,13 get v20 networks,2019-11-19 17:00:48.274000+01:00,,NaN,1
1,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:01:50.119000000+01:00,"130.149.249.127 ""GET /v2.0/ports?fields=bindin...",,fine_log_cluster_template#get v20 ports fields...,22 get v20 ports fields binding * id fields bi...,coarse_log_cluster_template#get v20 ports fiel...,22 get v20 ports,2019-11-19 17:01:50.119000+01:00,,NaN,1
2,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:01:50.210000000+01:00,"130.149.249.127 ""GET /v2.0/ports?tenant_id=99c...",,fine_log_cluster_template#get v20 ports tenant...,17 get v20 ports tenant id * device id *,coarse_log_cluster_template#get v20 ports tena...,17 get v20 ports,2019-11-19 17:01:50.210000+01:00,,NaN,1


In [81]:
log_df["all_events"] = log_df[sequence_preprocessor.relevant_columns].values.tolist()
log_df.head(3)

,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,fine_log_cluster_template,fine_log_cluster_path,coarse_log_cluster_template,coarse_log_cluster_path,timestamp,url_cluster_template,url_cluster_path,grouper,all_events
0,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:00:48.274000000+01:00,"130.149.249.127 ""GET /v2.0/networks?shared=Tru...",,fine_log_cluster_template#get v20 networks sha...,13 get v20 networks shared true http * status *,coarse_log_cluster_template#get v20 networks s...,13 get v20 networks,2019-11-19 17:00:48.274000+01:00,,NaN,1,"[, coarse_log_cluster_template#get v20 network..."
1,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:01:50.119000000+01:00,"130.149.249.127 ""GET /v2.0/ports?fields=bindin...",,fine_log_cluster_template#get v20 ports fields...,22 get v20 ports fields binding * id fields bi...,coarse_log_cluster_template#get v20 ports fiel...,22 get v20 ports,2019-11-19 17:01:50.119000+01:00,,NaN,1,"[, coarse_log_cluster_template#get v20 ports f..."
2,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:01:50.210000000+01:00,"130.149.249.127 ""GET /v2.0/ports?tenant_id=99c...",,fine_log_cluster_template#get v20 ports tenant...,17 get v20 ports tenant id * device id *,coarse_log_cluster_template#get v20 ports tena...,17 get v20 ports,2019-11-19 17:01:50.210000+01:00,,NaN,1,"[, coarse_log_cluster_template#get v20 ports t..."


In [82]:
log_df["attributes"] = log_df[[x for x in sequence_preprocessor.relevant_columns if not "log_cluster_template" in x]].values.tolist()
log_df.head(3)

,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,fine_log_cluster_template,fine_log_cluster_path,coarse_log_cluster_template,coarse_log_cluster_path,timestamp,url_cluster_template,url_cluster_path,grouper,all_events,attributes
0,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:00:48.274000000+01:00,"130.149.249.127 ""GET /v2.0/networks?shared=Tru...",,fine_log_cluster_template#get v20 networks sha...,13 get v20 networks shared true http * status *,coarse_log_cluster_template#get v20 networks s...,13 get v20 networks,2019-11-19 17:00:48.274000+01:00,,NaN,1,"[, coarse_log_cluster_template#get v20 network...","[, programname#neutron-server, , , python_modu..."
1,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:01:50.119000000+01:00,"130.149.249.127 ""GET /v2.0/ports?fields=bindin...",,fine_log_cluster_template#get v20 ports fields...,22 get v20 ports fields binding * id fields bi...,coarse_log_cluster_template#get v20 ports fiel...,22 get v20 ports,2019-11-19 17:01:50.119000+01:00,,NaN,1,"[, coarse_log_cluster_template#get v20 ports f...","[, programname#neutron-server, , , python_modu..."
2,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:01:50.210000000+01:00,"130.149.249.127 ""GET /v2.0/ports?tenant_id=99c...",,fine_log_cluster_template#get v20 ports tenant...,17 get v20 ports tenant id * device id *,coarse_log_cluster_template#get v20 ports tena...,17 get v20 ports,2019-11-19 17:01:50.210000+01:00,,NaN,1,"[, coarse_log_cluster_template#get v20 ports t...","[, programname#neutron-server, , , python_modu..."


In [83]:
for log_template_column in [x for x in sequence_preprocessor.relevant_columns if "log_cluster_template" in x]:
    log_df[log_template_column] = log_df[log_template_column].apply(lambda x: [x])
log_df.head(3)

,Hostname,log_level,programname,python_module,http_status,http_method,@timestamp,Payload,http_url,fine_log_cluster_template,fine_log_cluster_path,coarse_log_cluster_template,coarse_log_cluster_path,timestamp,url_cluster_template,url_cluster_path,grouper,all_events,attributes
0,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:00:48.274000000+01:00,"130.149.249.127 ""GET /v2.0/networks?shared=Tru...",,[fine_log_cluster_template#get v20 networks sh...,13 get v20 networks shared true http * status *,[coarse_log_cluster_template#get v20 networks ...,13 get v20 networks,2019-11-19 17:00:48.274000+01:00,,NaN,1,"[, coarse_log_cluster_template#get v20 network...","[, programname#neutron-server, , , python_modu..."
1,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:01:50.119000000+01:00,"130.149.249.127 ""GET /v2.0/ports?fields=bindin...",,[fine_log_cluster_template#get v20 ports field...,22 get v20 ports fields binding * id fields bi...,[coarse_log_cluster_template#get v20 ports fie...,22 get v20 ports,2019-11-19 17:01:50.119000+01:00,,NaN,1,"[, coarse_log_cluster_template#get v20 ports f...","[, programname#neutron-server, , , python_modu..."
2,Hostname#wally113,log_level#info,programname#neutron-server,python_module#neutron.wsgi,,,2019-11-19T17:01:50.210000000+01:00,"130.149.249.127 ""GET /v2.0/ports?tenant_id=99c...",,[fine_log_cluster_template#get v20 ports tenan...,17 get v20 ports tenant id * device id *,[coarse_log_cluster_template#get v20 ports ten...,17 get v20 ports,2019-11-19 17:01:50.210000+01:00,,NaN,1,"[, coarse_log_cluster_template#get v20 ports t...","[, programname#neutron-server, , , python_modu..."


In [84]:
events_per_trace = (
            log_df
            .groupby('grouper')
            .agg(
                {
                    column_name: lambda x: list(x)
                    for column_name in ["all_events", "attributes",]
                    + [x for x in sequence_preprocessor.relevant_columns if "log_cluster_template" in x]
                }
            )
            .reset_index()
        )
(events_per_trace['all_events'].iloc[0])[25][0] == (log_df['all_events'].iloc[25])[0]

True

In [85]:
events_per_trace['num_logs'] = events_per_trace[huawei_config.relevant_log_column].apply(
    lambda x: len([loglist for loglist in x if len(loglist[0]) > 0]))
events_per_trace

,grouper,all_events,attributes,coarse_log_cluster_template,fine_log_cluster_template,num_logs
0,1,"[[, coarse_log_cluster_template#get v20 networ...","[[, programname#neutron-server, , , python_mod...",[[coarse_log_cluster_template#get v20 networks...,[[fine_log_cluster_template#get v20 networks s...,50


In [86]:
events_per_trace['num_events'] = events_per_trace[huawei_config.relevant_log_column].apply(lambda x: len(x))
events_per_trace

,grouper,all_events,attributes,coarse_log_cluster_template,fine_log_cluster_template,num_logs,num_events
0,1,"[[, coarse_log_cluster_template#get v20 networ...","[[, programname#neutron-server, , , python_mod...",[[coarse_log_cluster_template#get v20 networks...,[[fine_log_cluster_template#get v20 networks s...,50,50


In [88]:
final_preprocessing_result = events_per_trace[events_per_trace.columns.drop('grouper')]
final_preprocessing_result

,all_events,attributes,coarse_log_cluster_template,fine_log_cluster_template,num_logs,num_events
0,"[[, coarse_log_cluster_template#get v20 networ...","[[, programname#neutron-server, , , python_mod...",[[coarse_log_cluster_template#get v20 networks...,[[fine_log_cluster_template#get v20 networks s...,50,50
